In [1]:
import warnings
warnings.filterwarnings(action='ignore')
%config Computer.use_jedi = False
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import missingno as msno
plt.rc('font', family='D2Coding')
from plotnine import *
import requests

In [2]:
subway = pd.read_csv('./data/서울지하철위도경도.csv', encoding='EUC-KR')
subway.head()

,연번,호선,고유역번호(외부역코드),역명,위도,경도,작성일자
0,1,1,150,서울,37.553150,126.972533,1974-02-28
1,2,1,151,시청,37.563590,126.975407,1974-08-15
2,3,1,152,종각,37.570203,126.983116,1974-08-15
3,4,1,153,종로3가,37.570429,126.992095,1974-08-15
4,5,1,154,종로5가,37.570971,127.001900,1974-03-31


In [3]:
# 네이버 개발자 포털에서 발급한 Client ID와 Client Secret
client_id = '5j924usqjd'
client_secret = 'W4wjDI2jRzuvP1N0zo9UiG1zTRoDP30idjpESbp4'

# '위도'와 '경도' 정보가 있는 데이터프레임 (house_df)에서 반복적으로 역 지오코딩을 수행합니다.
for index, row in subway.iterrows():
    latitude = row['위도']
    longitude = row['경도']
#     https://naveropenapi.apigw.ntruss.com/map-reversegeocode/v2/gc?request=coordsToaddr&coords=129.1133567,35.2982640&sourcecrs=epsg:4326&output=json&orders=addr,admcode
    if latitude is not None and longitude is not None:
        url = f'https://naveropenapi.apigw.ntruss.com/map-reversegeocode/v2/gc?request=coordsToaddr&coords={longitude},{latitude}&sourcecrs=epsg:4326&output=json&orders=addr,admcode'
        headers = {
            'X-NCP-APIGW-API-KEY-ID': client_id,
            'X-NCP-APIGW-API-KEY': client_secret
        }
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            data = response.json()
            if data.get('results'):
                gu = data['results'][0]['region']['area2']['name']  # 예시로 주소 중 'area3'의 'name' 사용
                dong = data['results'][0]['region']['area3']['name']  # 예시로 주소 중 'area3'의 'name' 사용
                subway.at[index, '법정동'] = dong  # 주소 정보를 데이터프레임에 추가
                subway.at[index, '시군구'] = gu  # 주소 정보를 데이터프레임에 추가
                
subway

,연번,호선,고유역번호(외부역코드),역명,위도,경도,작성일자,법정동,시군구
0,1,1,150,서울,37.553150,126.972533,1974-02-28,동자동,용산구
1,2,1,151,시청,37.563590,126.975407,1974-08-15,서소문동,중구
2,3,1,152,종각,37.570203,126.983116,1974-08-15,관철동,종로구
3,4,1,153,종로3가,37.570429,126.992095,1974-08-15,종로3가,종로구
4,5,1,154,종로5가,37.570971,127.001900,1974-03-31,종로5가,종로구
...,...,...,...,...,...,...,...,...,...
271,272,8,2823,남한산성입구,37.451568,127.159845,1996-10-31,금광동,성남시 중원구
272,273,8,2824,단대오거리,37.445057,127.156735,1996-12-28,신흥동,성남시 수정구
273,274,8,2825,신흥,37.440952,127.147590,1996-12-28,신흥동,성남시 수정구
274,275,8,2826,수진,37.437575,127.140936,1996-12-28,성남동,성남시 중원구


In [4]:
subway_dongbuk = subway[(subway['시군구'] == '강북구') | (subway['시군구'] == '도봉구') | (subway['시군구'] == '노원구')| (subway['시군구'] == '중랑구')
                            | (subway['시군구'] == '성북구')| (subway['시군구'] == '동대문구')]
subway_dongbuk

,연번,호선,고유역번호(외부역코드),역명,위도,경도,작성일자,법정동,시군구
7,8,1,156,신설동,37.576117,127.024710,1974-03-30,신설동,동대문구
8,9,1,157,제기동,37.578116,127.034902,1974-03-30,용두동,동대문구
9,10,1,158,청량리,37.580148,127.045063,1974-08-31,청량리동,동대문구
53,54,2,246,신설동,37.574653,127.025158,1977-08-29,신설동,동대문구
54,55,2,250,용두,37.574012,127.038110,2005-12-30,용두동,동대문구
95,96,4,409,당고개,37.669560,127.078404,1993-07-31,상계동,노원구
96,97,4,410,상계,37.660576,127.073199,1985-05-20,상계동,노원구
97,98,4,411,노원,37.656274,127.063183,1985-05-20,상계동,노원구
98,99,4,412,창동,37.652993,127.046746,1985-05-20,창동,도봉구
99,100,4,413,쌍문,37.648274,127.034381,1985-05-20,쌍문동,도봉구


In [5]:
subway_dongbuk.drop(columns=['연번', '호선', '고유역번호(외부역코드)','위도', '경도', '작성일자'], inplace=True)
subway_dongbuk

,역명,법정동,시군구
7,신설동,신설동,동대문구
8,제기동,용두동,동대문구
9,청량리,청량리동,동대문구
53,신설동,신설동,동대문구
54,용두,용두동,동대문구
95,당고개,상계동,노원구
96,상계,상계동,노원구
97,노원,상계동,노원구
98,창동,창동,도봉구
99,쌍문,쌍문동,도봉구


In [6]:
# 숫자 + '가'가 들어가는 법정동을 '시가지동'으로 변경
subway_dongbuk['법정동'] = subway_dongbuk['법정동'].apply(lambda x: '시가지동' if x.endswith('가')  or x == '돈암동' else x)

# 변경된 데이터프레임 출력
subway_dongbuk

,역명,법정동,시군구
7,신설동,신설동,동대문구
8,제기동,용두동,동대문구
9,청량리,청량리동,동대문구
53,신설동,신설동,동대문구
54,용두,용두동,동대문구
95,당고개,상계동,노원구
96,상계,상계동,노원구
97,노원,상계동,노원구
98,창동,창동,도봉구
99,쌍문,쌍문동,도봉구


In [7]:
# '시군구명'과 '법정동명'을 기준으로 그룹화하고 '법정동'의 데이터 수를 카운트
grouped = subway_dongbuk.groupby(['시군구', '법정동']).size().reset_index(name='역수')
grouped

,시군구,법정동,역수
0,강북구,미아동,2
1,강북구,번동,1
2,노원구,공릉동,3
3,노원구,상계동,6
4,노원구,중계동,1
5,노원구,하계동,1
6,도봉구,도봉동,1
7,도봉구,쌍문동,1
8,도봉구,창동,1
9,동대문구,신설동,2


In [8]:
grouped.to_csv('./data/동북부지하철수.csv')
grouped

,시군구,법정동,역수
0,강북구,미아동,2
1,강북구,번동,1
2,노원구,공릉동,3
3,노원구,상계동,6
4,노원구,중계동,1
5,노원구,하계동,1
6,도봉구,도봉동,1
7,도봉구,쌍문동,1
8,도봉구,창동,1
9,동대문구,신설동,2


In [9]:
width = pd.read_csv('./data/동북부면적.csv')
width = width.drop(columns=['Unnamed: 0'], axis=1)
width

,시군구,법정동,면적
0,동대문구,신설동,0.22
1,동대문구,용두동,0.76
2,동대문구,제기동,0.72
3,동대문구,전농동,0.94
4,동대문구,답십리동,1.02
5,동대문구,장안동,1.47
6,동대문구,청량리동,0.62
7,동대문구,회기동,0.28
8,동대문구,휘경동,0.86
9,동대문구,이문동,0.91
